In [19]:
import pandas as pd
import scanpy as sc
import json

In [20]:
datasets = ['PBMC1', 'PBMC2', 'PBMC3', 'PBMC4']
for dataset in datasets:
    dataset_path = f"../dataset/{dataset}-Filtered/10x/"
    adata = sc.read_10x_mtx(dataset_path, var_names="gene_symbols", cache=False)
    filtered_barcodes = adata.obs.index.to_list()
    labels = pd.read_csv(f"../dataset/{dataset}-Filtered/raw/celltypist_labels.csv", index_col=0)
    labels = labels[labels.index.isin(filtered_barcodes)]
    counts = labels.value_counts().to_frame()
    counts.index.names = ['id']
    mapping = pd.read_csv(f"../dataset/{dataset}-Filtered/raw/celltypist_mapping.csv", index_col=0)
    counts_mappings = counts.merge(mapping, on='id')
    counts_mappings.to_csv(f"../dataset/{dataset}-Filtered/raw/celltypist_counts_filtered.csv")
    with open(f'../dataset/{dataset}-Filtered/raw/nclusters.json', 'w') as fp:
        json.dump({'nclusters': counts_mappings[counts_mappings['count']>40].shape[0]}, fp)